In [168]:
import numpy as np
import pandas as pd

In [169]:
df_sellers = pd.read_csv('./Datasets/olist_sellers_dataset.csv')
df_costumers = pd.read_csv('./Datasets/olist_customers_dataset.csv')
df_geo = pd.read_csv('./Datasets/olist_geolocation_dataset.csv')

In [170]:
df_cep = pd.read_csv('./Datasets/TB_CEP_BR_2018.csv', delimiter=';', dtype='string')
df_post = pd.read_csv('./Datasets/postal_code.csv', delimiter = ';')
df_scraped = pd.read_csv('./Datasets/scraped_cep.csv', delimiter = ';')

In [171]:
def to_num(x):
    x = x[:-3]
    if x.isnumeric():
        return int(x)
    else:
        return x

In [172]:
df_cep = df_cep.loc[:,['CEP','CIDADE','UF']]
df_cep.rename(columns={'CIDADE': 'City', 'UF': 'State'}, inplace=True)

In [173]:
df_cep['CEP'] = df_cep['CEP'].apply(to_num)

In [174]:
df_zip = pd.concat([df_cep, df_post, df_scraped], ignore_index=True)

In [175]:
def get_city(x):
    city = x.split('(')
    if len(city) > 1:
        return city[1][:-1]
    else:
        return x

def get_state(x):
    state = x.split(' ')
    if len(state) > 1:
        return state[0]
    else:
        return x

In [176]:
df_zip['City'] = df_zip['City'].apply(get_city)
df_zip['State'] = df_zip['State'].apply(get_state)

In [177]:
df_zip.drop_duplicates(inplace=True)
df_zip.sort_values(by='CEP', inplace=True)
df_zip.reset_index(inplace=True, drop = True)

In [178]:
df_zip['City'] = df_zip['City'].apply(lambda x: np.nan if x == 'Null' else x)
df_zip['State'] = df_zip['State'].apply(lambda x: np.nan if x == 'Null' else x)

In [179]:
def fillna(x):
    if x.shift(1) == x.shift(-1):
        return x.shift(1)
    else:
        return x

In [180]:
df_zip['City'] = df_zip['City'].shift(1).where((df_zip['City'].isna()) & (df_zip['City'].shift(1) == df_zip['City'].shift(-1)),df_zip['City'])
df_zip['State'] = df_zip['State'].shift(1).where((df_zip['State'].isna()) & (df_zip['State'].shift(1) == df_zip['State'].shift(-1)),df_zip['State'])
df_zip.rename(columns={'CEP':'cep', 'City':'city', 'State':'state'}, inplace=True)
df_zip

,cep,city,state
0,1001,São Paulo,SP
1,1002,São Paulo,SP
2,1003,São Paulo,SP
3,1004,São Paulo,SP
4,1005,São Paulo,SP
...,...,...,...
24949,99970,Ciríaco,RS
24950,99975,Ciríaco,RS
24951,99978,Ciríaco,RS
24952,99980,David Canabarro,RS


In [181]:
df_cities = df_zip.loc[:,['city', 'state']]
df_cities.drop_duplicates(inplace=True)
df_cities = df_cities[~df_cities['city'].isna()]
df_cities.reset_index(drop=True, inplace=True)

In [182]:
df_cities.insert(0,'city_id', range(1,1+len(df_cities)))

In [183]:
df_cities

,city_id,city,state
0,1,São Paulo,SP
1,2,Osasco,SP
2,3,Carapicuíba,SP
3,4,Barueri,SP
4,5,Santana de Parnaíba,SP
...,...,...,...
5624,5625,Charrua,RS
5625,5626,Água Santa,RS
5626,5627,Ciríaco,RS
5627,5628,David Canabarro,RS


In [184]:
df_state = df_zip.loc[:,'state']
df_state.drop_duplicates(inplace=True)
df_state = df_state[~df_state.isna()]
df_state.reset_index(drop=True, inplace=True)
df_state = pd.DataFrame(data=df_state)
df_state.insert(0,'state_id', range(1,1+len(df_state)))

In [185]:
df_state.rename(columns={'state':'state_code'}, inplace=True)

In [186]:
estados_br_df = pd.read_html("https://es.wikipedia.org/wiki/Organizaci%C3%B3n_territorial_de_Brasil")
estados_br_df = estados_br_df[0]
estados_br_df.drop(['Número','Bandera','Escudo','Área (km²)','Población','Mapa'], axis=1, inplace=True)
estados_br_df.rename(columns={'Entidad federal':'nombre_estado', 'Capital':'capital'}, inplace=True)
estados_br_df.drop(estados_br_df.index[estados_br_df.loc[estados_br_df['nombre_estado'] == 'República Federal de Brasil'].index], inplace=True)
estados_br_df.insert(0,'sigla_estado',estados_br_df.nombre_estado.str[-3:-1])

In [187]:
estados_br_df.drop(columns=['capital'], inplace=True)
estados_br_df.rename(columns={'sigla_estado':'state_code', 'nombre_estado':'state_name'}, inplace=True)
estados_br_df

,state_code,state_name
0,AC,Acre (AC)
1,AL,Alagoas (AL)
2,AP,Amapá (AP)
3,AM,Amazonas (AM)
4,BA,Bahía (BA)
5,CE,Ceará (CE)
6,ES,Espírito Santo (ES)
7,GO,Goiás (GO)
8,MA,Maranhão (MA)
9,MT,Mato Grosso (MT)


In [188]:
estados_br_df['state_name'] = estados_br_df['state_name'].apply(lambda x: x.split('(')[0][:-1])
estados_br_df

,state_code,state_name
0,AC,Acre
1,AL,Alagoas
2,AP,Amapá
3,AM,Amazonas
4,BA,Bahía
5,CE,Ceará
6,ES,Espírito Santo
7,GO,Goiás
8,MA,Maranhão
9,MT,Mato Grosso


In [192]:
df_state = df_state.merge(right=estados_br_df, on='state_code')
df_state

,state_id,state_code,state_name
0,1,SP,São Paulo
1,2,RJ,Río de Janeiro
2,3,ES,Espírito Santo
3,4,MG,Minas Gerais
4,5,BA,Bahía
5,6,SE,Sergipe
6,7,PE,Pernambuco
7,8,AL,Alagoas
8,9,PB,Paraíba
9,10,RN,Río Grande del Norte


In [190]:
df_cities = df_cities.merge(right=df_state, on='state')
df_cities.drop(columns='state', inplace=True)
df_cities

KeyError: 'state'

In [ ]:
df_zip = df_zip.merge(right=df_cities, on='city')
df_zip.drop(columns=['city', 'state', 'state_id'], inplace=True)
df_zip